<a href="https://colab.research.google.com/github/aymuos/starship/blob/main/fundamental_EDA/actuallyWorking/feature_creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install duckdb polars pyarrow

## Objective

- create features using both the datasets
-

In [11]:
base = "/content/drive/MyDrive/ml/PROCESSED/matched/city_divided/"

idea : trajectory[t−Δ , t]  →  delivery(t) where t = receipt_time
Δ = 15 minutes (initial)

In [12]:
import polars as pl

cq_delivery = "/content/drive/MyDrive/ml/PROCESSED/matched/city_divided/chongqing_data.csv"
cq_gps = "/content/drive/MyDrive/ml/PROCESSED/matched/city_divided/chongqing_delivery_data.parquet"

# Load the CSV file
chongqing_df_csv = pl.read_csv(cq_delivery)
print("\nChongqing CSV DataFrame (head 5):")
print(chongqing_df_csv.head(5))

# Load the Parquet file
chongqing_df_parquet = pl.read_parquet(cq_gps)
print("\nChongqing Parquet DataFrame (head 5):")
print(chongqing_df_parquet.head(5))


Chongqing CSV DataFrame (head 5):
shape: (5, 16)
┌─────────────┬─────────────┬────────────┬──────────┬───┬──────────┬─────┬────────────┬────────────┐
│ order_id    ┆ from_dipan_ ┆ delivery_u ┆ poi_lng  ┆ … ┆ sign_lat ┆ ds  ┆ from_city_ ┆ horizon_ET │
│ ---         ┆ id          ┆ ser_id     ┆ ---      ┆   ┆ ---      ┆ --- ┆ en         ┆ A          │
│ str         ┆ ---         ┆ ---        ┆ f64      ┆   ┆ str      ┆ i64 ┆ ---        ┆ ---        │
│             ┆ str         ┆ str        ┆          ┆   ┆          ┆     ┆ str        ┆ f64        │
╞═════════════╪═════════════╪════════════╪══════════╪═══╪══════════╪═════╪════════════╪════════════╡
│ 55be8cdf127 ┆ c5ac5ba9980 ┆ df0b594618 ┆ 8.8999e6 ┆ … ┆ null     ┆ 318 ┆ Chongqing  ┆ 361.0      │
│ 0526231c9ba ┆ 1aa6b85ba47 ┆ d1ba6f619e ┆          ┆   ┆          ┆     ┆            ┆            │
│ 3387f51b…   ┆ 3d926051…   ┆ 4e7dd03444 ┆          ┆   ┆          ┆     ┆            ┆            │
│             ┆             ┆ …          

In [13]:
chongqing_df_csv = chongqing_df_csv.with_columns([
    pl.col("receipt_time").str.to_datetime("%Y-%m-%d %H:%M:%S").alias("receipt_time"),
    pl.col("sign_time").str.to_datetime("%Y-%m-%d %H:%M:%S").alias("sign_time")
])

print("Chongqing CSV DataFrame with updated time column types:")
print(chongqing_df_csv.head(5))

Chongqing CSV DataFrame with updated time column types:
shape: (5, 16)
┌─────────────┬─────────────┬────────────┬──────────┬───┬──────────┬─────┬────────────┬────────────┐
│ order_id    ┆ from_dipan_ ┆ delivery_u ┆ poi_lng  ┆ … ┆ sign_lat ┆ ds  ┆ from_city_ ┆ horizon_ET │
│ ---         ┆ id          ┆ ser_id     ┆ ---      ┆   ┆ ---      ┆ --- ┆ en         ┆ A          │
│ str         ┆ ---         ┆ ---        ┆ f64      ┆   ┆ str      ┆ i64 ┆ ---        ┆ ---        │
│             ┆ str         ┆ str        ┆          ┆   ┆          ┆     ┆ str        ┆ f64        │
╞═════════════╪═════════════╪════════════╪══════════╪═══╪══════════╪═════╪════════════╪════════════╡
│ 55be8cdf127 ┆ c5ac5ba9980 ┆ df0b594618 ┆ 8.8999e6 ┆ … ┆ null     ┆ 318 ┆ Chongqing  ┆ 361.0      │
│ 0526231c9ba ┆ 1aa6b85ba47 ┆ d1ba6f619e ┆          ┆   ┆          ┆     ┆            ┆            │
│ 3387f51b…   ┆ 3d926051…   ┆ 4e7dd03444 ┆          ┆   ┆          ┆     ┆            ┆            │
│             ┆     

In [14]:
print("Chongqing CSV DataFrame schema after datetime conversion:")
print(chongqing_df_csv.schema)

Chongqing CSV DataFrame schema after datetime conversion:
Schema({'order_id': String, 'from_dipan_id': String, 'delivery_user_id': String, 'poi_lng': Float64, 'poi_lat': Float64, 'aoi_id': String, 'typecode': String, 'receipt_time': Datetime(time_unit='us', time_zone=None), 'receipt_lng': Float64, 'receipt_lat': Float64, 'sign_time': Datetime(time_unit='us', time_zone=None), 'sign_lng': String, 'sign_lat': String, 'ds': Int64, 'from_city_en': String, 'horizon_ETA': Float64})


In [17]:
chongqing_df_csv = chongqing_df_csv.drop(["sign_lat", "sign_lng"])

print("Chongqing CSV DataFrame after dropping sign_lat and sign_lng (head 5):")
print(chongqing_df_csv.head(5))

print("\nChongqing CSV DataFrame schema after dropping sign_lat and sign_lng:")
print(chongqing_df_csv.schema)

ColumnNotFoundError: "sign_lng" not found

Feature creation starts
---


---


In [18]:
delivery = (
    chongqing_df_csv
    .rename({"horizon_ETA": "eta_seconds"})
    .with_columns(
        pl.col("delivery_user_id").cast(pl.Utf8)
    )
    .sort(["delivery_user_id", "receipt_time"])
)

In [19]:
# defining just the gps extraction windows

start_time = delivery["receipt_time"].min()
end_time   = delivery["receipt_time"].max()

couriers = delivery["delivery_user_id"].unique().to_list()

print(len(couriers), "couriers")

145 couriers


In [21]:
import duckdb

con = duckdb.connect()

PRE_MIN = 15

gps_df = con.execute(f"""
SELECT
    postman_id,
    gps_time,
    lat,
    lng
FROM parquet_scan('/content/drive/MyDrive/ml/PROCESSED/matched/city_divided/chongqing_delivery_data.parquet')
WHERE
    postman_id IN {tuple(couriers)}
    AND gps_time BETWEEN
        TIMESTAMP '{start_time}' - INTERVAL '{PRE_MIN} minutes'
        AND TIMESTAMP '{end_time}'
""").pl()

gps_df = gps_df.rename({"postman_id": "delivery_user_id"})
gps_df = gps_df.sort(["delivery_user_id", "gps_time"])

In [22]:
# Courier Snapshot Before Delivery (ASOF JOIN)

state = delivery.join_asof(
    gps_df,
    left_on="receipt_time",
    right_on="gps_time",
    by="delivery_user_id",
    strategy="backward"
).rename({
    "lat": "last_x",
    "lng": "last_y",
    "gps_time": "last_gps_time"
})

/tmp/ipython-input-950023143.py:3: UserWarning: Sortedness of columns cannot be checked when 'by' groups provided
  state = delivery.join_asof(


In [24]:
# GPS Freshness Feature
state = state.with_columns(
    (pl.col("receipt_time") - pl.col("last_gps_time"))
        .dt.total_minutes()
        .alias("gps_gap_min")
)

In [25]:
state.select("gps_gap_min").describe()

statistic,gps_gap_min
str,f64
"""count""",19062.0
"""null_count""",7608.0
"""mean""",315.771168
"""std""",1520.738809
"""min""",0.0
"""25%""",0.0
"""50%""",0.0
"""75%""",0.0
"""max""",15808.0


gps_time ≤ receipt_time is ensured by asof_join . Invalidate stale gps matches : GPS older than 30 minutes → unusable

-

In [26]:
MAX_GAP = 30  # minutes

state = state.with_columns(
    pl.when(pl.col("gps_gap_min") <= MAX_GAP)
      .then(pl.col("gps_gap_min"))
      .otherwise(None)
      .alias("gps_gap_min")
)

In [27]:
state = state.with_columns([
    pl.when(pl.col("gps_gap_min").is_null())
      .then(None)
      .otherwise(pl.col("last_x"))
      .alias("last_x"),

    pl.when(pl.col("gps_gap_min").is_null())
      .then(None)
      .otherwise(pl.col("last_y"))
      .alias("last_y"),
])


# idea is to prevent stale gps from corrupting matches

In lade Delivery coverage > GPS coverage , So GPS must be treated as:
optional contextual sensor not guaranteed observation. This will influence later bandit formulation.

In [29]:
# diagonistic code

state.select([
    pl.len().alias("rows"),
    pl.col("gps_gap_min").null_count().alias("missing_gps"),
    pl.mean("gps_gap_min").alias("mean_gap"),
    pl.median("gps_gap_min").alias("median_gap"),
])

rows,missing_gps,mean_gap,median_gap
u32,u32,f64,f64
26670,9472,0.324689,0.0


This means :

- rows          = 26670 deliveries
- missing_gps   = 9472  (~35%)
- mean_gap      = 0.32 minutes
- median_gap    = 0.0 minutes

---

- Most deliveries have a GPS sample almost exactly at receipt_time.
- mean_gap < 1 minute i.e stale gps are removed


# Building trajectory window